# <center style="font-family: consolas; font-size: 32px; font-weight: bold;">  Hands-On LangChain for LLM Applications Development: Prompt Templates </center>

# <center style="font-family: consolas; font-size: 25px; font-weight: bold;">  Understanding LangChain Prompt Templates   </center>
***

By prompting an LLM or large language model, it is possible to develop complex AI applications much faster than ever before. However, an application can require prompting an LLM multiple times and parsing its output, so a lot of glue code must be written.

LangChain makes this development process much easier by using an easy set of abstractions to do this type of operation and by providing prompt templates. In this notebook, we will cover prompt templates, why it is important, and how to use them effectively, explained with practical examples.


#### <a id="top"></a>
# <div style="box-shadow: rgb(60, 121, 245) 0px 0px 0px 3px inset, rgb(255, 255, 255) 10px -10px 0px -3px, rgb(31, 193, 27) 10px -10px, rgb(255, 255, 255) 20px -20px 0px -3px, rgb(255, 217, 19) 20px -20px, rgb(255, 255, 255) 30px -30px 0px -3px, rgb(255, 156, 85) 30px -30px, rgb(255, 255, 255) 40px -40px 0px -3px, rgb(255, 85, 85) 40px -40px; padding:20px; margin-right: 40px; font-size:30px; font-family: consolas; text-align:center; display:fill; border-radius:15px; color:rgb(60, 121, 245);"><b>Table of contents</b></div>

<div style="background-color: rgba(60, 121, 245, 0.03); padding:30px; font-size:15px; font-family: consolas;">
<ul>
    <li><a href="#1" target="_self" rel=" noreferrer nofollow">1. Setting Up Working Environment & Getting Started </a> </li>
    <li><a href="#2" target="_self" rel=" noreferrer nofollow">2. Prompt Template using LangChain </a></li>
    <li><a href="#3" target="_self" rel=" noreferrer nofollow">3. Why do We Need LangChain Prompt Templates? </a></li> 
</ul>
</div>

***



# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Setting Up Working Environment & Getting Started </b></div>


To get started we are going to import OS, import OpenAI, and load my OpenAI secret key. If you’re running this locally and don’t have OpenAI installed yet, you might need to run pip to install OpenAI.

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
import openai
import os
from google.colab import userdata

openai.api_key=userdata.get('api_key')
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai.api_key,
)

After that, we will define the LLM model that we will use which will be gpt-3.5-turbo.

In [ ]:
llm_model = "gpt-3.5-turbo"

Next, we will define a helper function that will take the input prompt and return the response.

In [ ]:
def get_completion(prompt, 
                   llm_model=llm_model, 
                   temperature=0, 
                   max_tokens=500):

    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=llm_model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content


Let’s start with an example where you get an email from a customer in a language other than formal English. To make sure the example is understandable, the other language we will use is the English pirate language:


In [ ]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

We will ask the LLM to translate the text to formal English in a calm and respectful tone. I will set the style to American English in a calm and respectful tone. I will specify the prompt using an f-string with the instructions, translate the text that is delimited by triple backticks into style, and then plug in these two styles. This generates a prompt that says translate the text.

In [ ]:
style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Let’s see what the response is:

In [ ]:
response = get_completion(prompt)
response

That sounds very nice and calm. Therefore if you have different customers writing reviews in different languages, not just English pirates, but French, German, Japanese, and so on, you can imagine having to generate a whole sequence of prompts to generate such translations. Let’s look at how we can do this in a more convenient, way using **LangChain**.


<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Prompt Template using LangChain </b></div>



Let's start with importing chat OpenAI. This is LangChain’s abstraction for the chatGPT API endpoint. I will set the temperature parameter to be equal to zero to make the output a little bit less random.

In [ ]:
!pip install pip install langchain-community

In [ ]:
from langchain_community.chat_models import ChatOpenAI

# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0

chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=openai.api_key)

We will define the template string as follows. Translate the text delimited by triple vectors into a style that is style, and then here’s the text.

In [ ]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

To repeatedly reuse this template, we have to import LangChain’s chat prompt template, and then, let me create a prompt template using that template string that we just wrote above.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


From the prompt template, you can extract the original prompt, and it realizes that this prompt has two input variables, the style, and the text, shown here with the curly braces.

In [ ]:
prompt_template.messages[0].prompt


We can also print the input variables out, and you can see that it realizes it has two input variables the style and text

In [ ]:
prompt_template.messages[0].prompt.input_variables


Now, let’s specify the style. This is a style that I want the customer message to be translated to, so I’m going to call this customer style, and here’s my same customer email as before.

In [ ]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""


If I create customer messages, this will generate the prompt and will pass this large language model to get a response.

If you want to look at the types, the customer message is a list, and if you look at the first element of the list, this is more or less the prompt that you would expect this to be creating.


In [ ]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))

Lastly, let’s pass this prompt to the LLM, so I’m going to call chat, which we had set earlier, as a reference to the OpenAI chatGPT endpoint, and, if we print out the customer responses content, then, it gives you back this text translated from English pirate to polite American English

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)

print(customer_response.content)

Of course, you can imagine other use cases where the customer emails are in other languages and this too can be used to translate the messages for an English-speaking to understand and reply to.

So let’s say, an English-speaking customer service agent writes this and says,

In [ ]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

But let’s say this is what a customer service agent wants. We are going to specify that the service message is going to be translated to this pirate style. So we want it to be in a polite tone that speaks in English pirate. And because we previously created that prompt template, the cool thing is, that we can now reuse that prompt template and specify that the output style we want is this service style pirate and the text is this service reply.



In [ ]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""


And if we do that, that’s the prompt.

In [ ]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

And if we prompt, ChatGPT, this is the response it gives us back

In [ ]:
service_response = chat(service_messages)
print(service_response.content)

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Why do We Need LangChain Prompt Templates? </b></div>



We are using prompt templates instead of just an f-string prompt because as you build sophisticated applications, prompts can be quite long and detailed. Prompt templates are a useful abstraction to help you reuse good prompts when you can.

This is an example of a relatively long prompt to decide if a student’s solution is correct or not. And a prompt like this can be quite long, in which you can ask the LLM to first solve the problem, and then have the output in a certain format, and the output in a certain format.

In [ ]:
prompt = f"""
Determine if the student's solution is correct or not.

Question:
I'm building a solar power installation and I need \
 help working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \ 
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations 
as a function of the number of square feet.

Student's Solution:
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

LangChain prompt makes it easier to reuse a prompt like this. Also, as you will see in the next articles LangChain provides prompts for some common operations, such as summarization, question answering, connecting to SQL databases, or connecting to different APIs. So by using some of LangChain’s built-in prompts, you can quickly get an application working without needing to, engineer your prompts.

# <div style="box-shadow: rgba(240, 46, 170, 0.4) -5px 5px inset, rgba(240, 46, 170, 0.3) -10px 10px inset, rgba(240, 46, 170, 0.2) -15px 15px inset, rgba(240, 46, 170, 0.1) -20px 20px inset, rgba(240, 46, 170, 0.05) -25px 25px inset; padding:20px; font-size:30px; font-family: consolas; display:fill; border-radius:15px; color: rgba(240, 46, 170, 0.7)"> <b> ༼⁠ ⁠つ⁠ ⁠◕⁠‿⁠◕⁠ ⁠༽⁠つ Thank You!</b></div>

<p style="font-family:verdana; color:rgb(34, 34, 34); font-family: consolas; font-size: 16px;"> 💌 Thank you for taking the time to read through my notebook. I hope you found it interesting and informative. If you have any feedback or suggestions for improvement, please don't hesitate to let me know in the comments. <br><br> 🚀 If you liked this notebook, please consider upvoting it so that others can discover it too. Your support means a lot to me, and it helps to motivate me to create more content in the future. <br><br> ❤️ Once again, thank you for your support, and I hope to see you again soon!</p>